In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
!unzip -q "/content/drive/My Drive/Colab Notebooks/Emotion_detection/face-expression-recognition-dataset.zip"

In [4]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [5]:
from __future__ import print_function

In [6]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

In [7]:
num_classes = 7
img_rows,img_cols = 48,48
batch_size = 32

In [8]:
train_data_dir = '/content/images/train'
validation_data_dir = '/content/images/validation'

In [14]:
train_datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    rotation_range=30,
                                    shear_range=0.3,
                                    zoom_range=0.3,
                                    width_shift_range=0.4,
                                    height_shift_range=0.4,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

In [15]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train_generator = train_datagen.flow_from_directory(
					train_data_dir,
					color_mode='grayscale',
					target_size=(img_rows,img_cols),
					batch_size=batch_size,
					class_mode='categorical',
					shuffle=True)

Found 28821 images belonging to 7 classes.


In [59]:
validation_generator = validation_datagen.flow_from_directory(
							validation_data_dir,
							color_mode='grayscale',
							target_size=(img_rows,img_cols),
							batch_size=batch_size,
							class_mode='categorical',
							shuffle=True)

Found 7066 images belonging to 7 classes.


In [62]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding='same',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [63]:
model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


In [64]:
model.add(Conv2D(128,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [65]:
model.add(Conv2D(256,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [66]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))


In [67]:
model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

In [68]:
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [69]:
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
activation_39 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_36 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_40 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_37 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 24, 24, 32)       

In [70]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [32]:
# checkpoint = ModelCheckpoint('Emotion_detection_vgg.h5',
#                              monitor='val_loss',
#                              mode='min',
#                              save_best_only=True,
#                              verbose=1)

# earlystop = EarlyStopping(monitor='val_loss',
#                           min_delta=0,
#                           patience=3,
#                           verbose=1,
#                           restore_best_weights=True
#                           )

# reduce_lr = ReduceLROnPlateau(monitor='val_loss',
#                               factor=0.2,
#                               patience=3,
#                               verbose=1,
#                               min_delta=0.0001)

In [33]:
# callbacks = [earlystop,checkpoint,reduce_lr]


In [71]:

model.compile(loss='categorical_crossentropy',
              optimizer = 'Adam',
              metrics=['accuracy'])

In [72]:
train_samples = 28821
validation_samples = 7066
epochs=75

out=model.fit_generator(
                train_generator,
                steps_per_epoch = train_samples//batch_size,
                epochs = epochs,
                validation_data = validation_generator,
                validation_steps = validation_samples//batch_size)


Epoch 1/75
900/900 [==============================] - 38s 42ms/step - loss: 1.9602 - accuracy: 0.2095 - val_loss: 1.7698 - val_accuracy: 0.2678
Epoch 2/75
900/900 [==============================] - 37s 42ms/step - loss: 1.8161 - accuracy: 0.2371 - val_loss: 1.8162 - val_accuracy: 0.2594
Epoch 3/75
900/900 [==============================] - 37s 41ms/step - loss: 1.7943 - accuracy: 0.2485 - val_loss: 1.7470 - val_accuracy: 0.2781
Epoch 4/75
900/900 [==============================] - 37s 41ms/step - loss: 1.7552 - accuracy: 0.2736 - val_loss: 1.7783 - val_accuracy: 0.2767
Epoch 5/75
900/900 [==============================] - 37s 42ms/step - loss: 1.6705 - accuracy: 0.3294 - val_loss: 1.8034 - val_accuracy: 0.3128
Epoch 6/75
900/900 [==============================] - 37s 41ms/step - loss: 1.5832 - accuracy: 0.3764 - val_loss: 1.4676 - val_accuracy: 0.4182
Epoch 7/75
900/900 [==============================] - 37s 41ms/step - loss: 1.5215 - accuracy: 0.4062 - val_loss: 1.3514 - val_accuracy:

In [42]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [45]:
model.save('model.h5')    
model_file = drive.CreateFile({'title' : 'model.h5'})
model_file.SetContentFile('model.h5')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1PPTYit-10zYcJeikVJp4mbyQiTdwHzZS'})

In [77]:
import cv2
def prepare(filepath):
  IMG_SIZE = 48
  img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
  new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
  return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [89]:
prediction = model.predict([prepare("/content/drive/My Drive/Colab Notebooks/Emotion_detection/happy.jpg")])

In [79]:
CATEGORIES = ["happy","angry","fear","sad","surprise","neutral","disgust"]

In [90]:
print(CATEGORIES[int(prediction[0][0])])

happy
